# Face Verification using SiameseNet

### Importing libraries

In [2]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import uuid

import tensorflow as tf

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

### Setting up GPU growth limit

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

### Setting up paths

In [6]:
#Setting up paths 
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

LFW_PATH = 'lfw-deepfunneled'

#Creating folder structures
try:
    os.makedirs(POS_PATH)
    os.makedirs(NEG_PATH)
    os.makedirs(ANC_PATH)
except FileExistsError:
    print("Directories already exist!")

Directories already exist!


### Fetching LFW (*Labeled Faces in the Wild*) image data

In [ ]:
#Fetching negatives (LFW Dataset)
!tar -xf lfw-deepfunneled/lfw-deepfunneled.tar

tar: Error opening archive: Failed to open 'lfw-deepfunneled/lfw-deepfunneled.tar'


In [12]:
#Move LFW Images to Negatives 
try:
  for directory in os.listdir(LFW_PATH):
    for filee in os.listdir(os.path.join(LFW_PATH, directory)):
      EX_PATH = os.path.join(LFW_PATH, directory, filee)
      NEW_PATH = os.path.join(NEG_PATH, filee)
      os.replace(EX_PATH, NEW_PATH)

    print(f"Successfully moved LFW images to {NEG_PATH}.")
except:
  print("An error occured!")

Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\negative.
Successfully moved LFW images to data\ne

The LFW dataset is a dataset that consists of a large number of images containing faces. In the context of this project, LFW images will be used as the Negative components for trianing the Siamese Network.

### Collecting Anchor and Positive images (CV2)

In [30]:
#Collect positive and anchor classes using webcam
#Establish connection to webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
  #Read results
  ret, frame = cap.read()
  #Clipping frame to relevant size (250x250)
  frame = frame[120:120+250, 200:200+250]
  #Collecting anchors
  if cv2.waitKey(1) & 0XFF == ord('a'):
    #Create unique file path
    imgname = os.path.join(ANC_PATH, f'{uuid.uuid1()}.jpg')
    #Write anchor image to folder
    cv2.imwrite(imgname, frame)

  #Collecting positives
  if cv2.waitKey(1) & 0XFF == ord('p'):
    #Create unique file path
    imgname = os.path.join(POS_PATH, f'{uuid.uuid1()}.jpg')
    #Write anchor image to folder
    cv2.imwrite(imgname, frame)

  #Show image back to screen
  cv2.imshow('Image Collection', frame)
  
  #Breaking if appropriate key is pressed
  if cv2.waitKey(1) & 0XFF == ord('q'):
    break

#Release webcam
cap.release()
#Close .imshow() frame
cv2.destroyAllWindows

<function destroyAllWindows>

Upon executing the above code, a webcam window will appear and allow us to capture and store an Anchor image by pressing the ```a``` key on the keyboard. Similarly, by pressing ```p```, we can capture a Positive image. These images will be essential for the training of the Siamese Net.

In this project, we will use around 300 Anchor and 300 Positive images, but we will take more just in case.

Finally, pressing ```q``` allows us to close the webcam window.

#### Anchor Image (example)

![Anchor](data\anchor\ad8ff2a0-1ea5-11f0-9e77-04ed33d22284.jpg)

#### Positive Image (example)

![positive](data/positive/2f3d5e47-1ea6-11f0-a787-04ed33d22284.jpg)

Absolutely beautiful.

### Loading and Preprocessing Images

In [ ]:
#Creating tf.Dataset objects
anchor_data = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(300)
positive_data = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(300)
negative_data = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(300)

In [64]:
#Defining image preprocessing function 
def preprocess(file_path):
    #Loading
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    #Resizing
    img = tf.image.resize(img, (100, 100))
    #Scaling
    img = img / 255.0
    return img

def preprocess_twin(input_img, val_img, label):
    return preprocess(input_img), preprocess(val_img), label

### Creating Labeled Dataset

In [76]:
positives = tf.data.Dataset.zip((anchor_data, positive_data, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor_data)))))
negatives = tf.data.Dataset.zip((anchor_data, negative_data, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor_data)))))
data = positives.concatenate(negatives)

### Data Pipeline and Data Split

In [77]:
data = data.map(preprocess_twin).cache().shuffle(buffer_size=1024)
train_data = data.take(round(len(data)*0.7)).batch(16).prefetch(tf.data.AUTOTUNE)
test_data = data.skip(round(len(data)*0.7)).take(round(len(data)*0.3)).batch(16).prefetch(tf.data.AUTOTUNE)